<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/evaluation/probe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [1]:
!pip install -q transformers

In [2]:
!pip install -q sentencepiece

In [3]:
import os
import numpy as np
import pandas as pd

import torch

from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

2022-11-22 19:07:10.795772: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [23]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Black

In [24]:
!pip install black[jupyter]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 17.6 MB/s 
     |████████████████████████████████| 843 kB 43.8 MB/s 
     |████████████████████████████████| 96 kB 2.8 MB/s 
     |████████████████████████████████| 1.6 MB 54.9 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [ ]:
!black "/content/drive/MyDrive/Colab Notebooks/probe.ipynb"

### Probe Class

A probe instance can be used to generate predictions from any of our models.

It loads (and caches) appropriate tokenizers and models as needed.

The only method you need to use is the predict method.

In [4]:
class Probe:
    """
    A class used to represent an Awesome Puppies Question Generation Probe
    ...

    Attributes
    ----------
    models : dict
        a dictionary of cached models
    tokenizers : dict
        a dictionary of cached tokenizers

    Methods
    -------
    predict(context, answer, base_model='bart', training_dataset='amalgam',
            num_beams=1, early_stopping=False, no_repeat_ngram_size=0,
            maximum_input_length=1024, maximum_target_length=50)

        returns a prediction string

    retrieve_tokenizer (base_model='bart')

        returns and caches a tokenizer

    retrieve_model(base_model='bart', training_dataset='amalgam')

        returns and caches a model
    """

#    project_root = "/content/drive/MyDrive/w266 NLP Final Project/"
    project_root = "/home/localadmin/Documents/w266_NLP_Final_Project/"
    model_root = project_root + "Models/"

    def __init__(self):
        self.models = {}
        self.tokenizers = {}

    def predict(
        self,
        context,
        answer,
        base_model="bart",
        training_dataset="amalgam",
        num_beams=1,
        early_stopping=False,
        no_repeat_ngram_size=0,
        maximum_input_length=1024,
        maximum_target_length=50,
    ):

        tokenizer = self.retrieve_tokenizer(base_model)
        model = self.retrieve_model(base_model, training_dataset)

        if base_model == "bart":
            prompt_string = f"{answer} </s> {context}"
        elif base_model == "T5":
            prompt_string = f"generate question: answer: {answer} context: {context}"
        else:
            raise ValueError("invalid base model")

        inputs = tokenizer(
            prompt_string,
            return_tensors="pt",
            max_length=maximum_input_length,
            truncation=True,
            padding=True,
        )
        output_ids = model.generate(
            inputs["input_ids"].cuda(),
            max_length=maximum_target_length,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=early_stopping,
        )
        prediction = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        return prediction

    def retrieve_tokenizer(self, base_model="bart"):
        tokenizer = self.tokenizers.get(base_model)
        if tokenizer:
            return tokenizer

        if base_model == "bart":
            tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
        elif base_model == "T5":
            tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")
        else:
            raise ValueError("invalid base model")

        self.tokenizers[base_model] = tokenizer
        return tokenizer

    def retrieve_model(self, base_model="bart", training_dataset="amalgam"):
        model_tuple = (base_model, training_dataset)
        model = self.models.get(model_tuple)
        if model:
            return model

        model_dir = f"{self.model_root}{base_model}_base_pt_long.{training_dataset}"

        if base_model == "bart":
            print(f"Loading: {model_dir}")
            model = BartForConditionalGeneration.from_pretrained(model_dir)
        elif base_model == "T5":
            print(f"Loading: {model_dir}")
            model = T5ForConditionalGeneration.from_pretrained(model_dir)
        else:
            raise ValueError("invalid base model")

        model.to(torch.device("cuda:0"))
        self.models[model_tuple] = model
        return model

### Play Time

In [8]:
help(Probe)

Help on class Probe in module __main__:

class Probe(builtins.object)
 |  A class used to represent an Awesome Puppies Question Generation Probe
 |  ...
 |  
 |  Attributes
 |  ----------
 |  models : dict
 |      a dictionary of cached models
 |  tokenizers : dict
 |      a dictionary of cached tokenizers
 |  
 |  Methods
 |  -------
 |  predict(context, answer, base_model='bart', training_dataset='amalgam',
 |          num_beams=1, early_stopping=False, no_repeat_ngram_size=0,
 |          maximum_input_length=1024, maximum_target_length=50)
 |  
 |      returns a prediction string
 |  
 |  retrieve_tokenizer (base_model='bart')
 |  
 |      returns and caches a tokenizer
 |  
 |  retrieve_model(base_model='bart', training_dataset='amalgam')
 |  
 |      returns and caches a model
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  predict(self, context, answer, base_model='bart', training_dataset='amal

In [9]:
p = Probe()

In [7]:
context = "Mary had a little lamb, its fleece was white as snow. Everywhere the child went, the little lamb was sure to go."
answer = "Mary"

In [10]:
p.predict(context, answer)

Loading: /home/localadmin/Documents/w266_NLP_Final_Project/Models/bart_base_pt_long.amalgam


['who had a little lamb in her hair']

In [11]:
p.predict(context, answer, base_model="T5")

Loading: /home/localadmin/Documents/w266_NLP_Final_Project/Models/T5_base_pt_long.amalgam


['who had a little lamb in the bible']

In [12]:
p.predict(context, answer, maximum_target_length=8)

['who had a little lamb']

In [13]:
p.predict(context, answer, maximum_target_length=7, base_model="T5")

['who had a little lamb']

In [ ]:
p.predict(
    context,
    answer,
    maximum_target_length=8,
    base_model="bart",
    training_dataset="squad",
)

Loading: /content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_long.squad


['Who had a little lamb']

In [ ]:
list(p.models.keys())

[('bart', 'amalgam'), ('T5', 'amalgam'), ('bart', 'squad')]

In [14]:
context = """Once upon a time there were three bears who lived in a house in the forest. 
There was a great big father bear, a middle-sized mother bear and a tiny baby bear.
One morning, their breakfast porridge was too hot to eat, so they decided to go for a walk in the forest. 
While they were out, a little girl called Goldilocks came through the trees and found their house. 
She knocked on the door and, as there was no answer, she pushed it open and went inside.
In front of her was a table with three chairs, one large chair, one middle-sized chair and one small chair. 
On the table were three bowls of porridge, one large bowl, one middle-sized bowl and one small bowl – and three spoons.
Goldilocks was hungry and the porridge looked good, so she sat in the great big chair, picked up the large spoon and tried some of the porridge from the big bowl. 
But the chair was very big and very hard, the spoon was heavy and the porridge too hot.
Goldilocks jumped off quickly and went over to the middle-sized chair. 
But this chair was far too soft, and when she tried the porridge from the middle-sized bowl it was too cold. 
So she went over to the little chair and picked up the smallest spoon and tried some of the porridge from the tiny bowl.
This time it was neither too hot nor too cold. 
It was just right and so delicious that she ate it all up. 
But she was too heavy for the little chair and it broke in pieces under her weight.
Next Goldilocks went upstairs, where she found three beds. 
There was a great big bed, a middle-sized bed and a tiny little bed. 
By now she was feeling rather tired. 
So she climbed into the big bed and lay down. 
The big bed was very hard and far too big. 
Then she tried the middle-sized bed, but that was far too soft. 
So she climbed into the tiny little bed. 
It was neither too hard nor too soft. 
In fact, it felt just right, all cosy and warm. 
And in no time at all Goldilocks fell fast asleep.
In a little while, the three bears came back from their walk in the forest. 
They saw at once that someone had pushed open the door of their house and Father Bear looked around. 
Then roared with a growly voice.
Mother Bear said in a quiet gentle voice.
"Somebody has been sitting in my chair.
Then Little Bear said in small squeaky baby voice.
"Somebody has been sitting in my chair and has broken it!"
Then Father Bear looked at his bowl of porridge and saw the spoon in it and he said in his great big growly voice,
"SOMEBODY HAS BEEN EATING MY PORRIDGE"
Then Mother Bear saw that her bowl had a spoon in it, and said in her quiet voice.
"Somebody has been eating my porridge Little Bear looked at his porridge bowl and said in his small squeaky baby voice,
"Somebody has been eating my porridge, and has eaten it all up:-
Then the three bears went upstairs, and Father Bear saw at once that his bed was untidy, and he said in his great big growly voice,
"SOMEBODY HAS BEEN SLEEPING IN MY BED!"
Mother Bear saw that her bed, too, had the bedclothes turned back, and she said in her quiet gentle voice,
"Somebody has been sleeping in my bed!"
Then Little Bear looked at his bed and said in his small squeaky baby voice,
"Somebody is sleeping in my bed!"
He squeaked so loudly that Goldilocks woke up with a start. She jumped out of bed, and away she ran, down the stairs and out into the forest. And the three bears never saw her again."""


In [15]:
p.predict(context, "The forest")

['Where did Goldilocks live?']

In [16]:
p.predict(context, "It broke into pieces.")

['What happened to the little chair?']

In [17]:
p.predict(context, "Never again")

['How long did she stay in the house?']

In [ ]:
p.predict(context, "Too hot")

['Why did Goldilocks decide to eat porridge?']

In [ ]:
p.predict(context, "It was too hard")

['Why did Goldilocks want to sit in the big bed?']

In [ ]:
p.predict(context, "squeaky")

['What kind of voice did the bear have?']

In [19]:
context = """BERKELEY - Jackson Sirmon scored on a 37-yard return following a wild play with two fumbles and the Cal football team rallied from a double-digit deficit in the fourth quarter to beat Stanford 27-20 in the 125th edition of the Big Game on Saturday.
"It's really neat... as a defensive player, you don't get a lot of opportunities to score touchdowns and you have to enjoy them when you get them," said Sirmon, who played in his first Big Game on Saturday. "It was very neat especially considering the timing of it all.""
The Golden Bears (4-7, 2-6 Pac-12) trailed 17-6 in the fourth quarter when Jack Plummer started the comeback with a 1-yard touchdown pass to Monroe Young.
Then things really went crazy on the ensuing possession for Stanford (3-8, 1-8).
Backup quarterback Ashton Daniels took a direct snap and ran to his left. He was stripped of the ball by Daniel Scott and Cal's Jeremiah Earby recovered. But Daniels then knocked the ball loose from Earby for a second fumble on the play only to have Sirmon scoop it up and run in to give Cal a 20-17 lead with 9:54 to play.
"It bounced right to me,'' Sirmon said." I was in the right place at the right time. All I did that play is I didn't mess it up. The ball came right to me and I ran with it.
Stanford had three more drives to rally. But the Cardinal punted twice and Scott came up with an interception with just more than two minutes to play, setting up Jaydn Ott's 1-yard run that sealed the win in front of a sellout crowd of 51,892.
"We know how important this game is to us as a team, our administration and our support staff," Travers Family Head Football Coach Justin Wilcox said. "Our fans and students were just incredible tonight. What an awesome environment." """
 

In [20]:
import pprint

In [21]:
pprint.pprint(context)

('BERKELEY - Jackson Sirmon scored on a 37-yard return following a wild play '
 'with two fumbles and the Cal football team rallied from a double-digit '
 'deficit in the fourth quarter to beat Stanford 27-20 in the 125th edition of '
 'the Big Game on Saturday.\n'
 '"It\'s really neat... as a defensive player, you don\'t get a lot of '
 'opportunities to score touchdowns and you have to enjoy them when you get '
 'them," said Sirmon, who played in his first Big Game on Saturday. "It was '
 'very neat especially considering the timing of it all.""\n'
 'The Golden Bears (4-7, 2-6 Pac-12) trailed 17-6 in the fourth quarter when '
 'Jack Plummer started the comeback with a 1-yard touchdown pass to Monroe '
 'Young.\n'
 'Then things really went crazy on the ensuing possession for Stanford (3-8, '
 '1-8).\n'
 'Backup quarterback Ashton Daniels took a direct snap and ran to his left. He '
 "was stripped of the ball by Daniel Scott and Cal's Jeremiah Earby recovered. "
 'But Daniels then knoc

In [22]:
p.predict(context, "Jakson Sirmon")

['Who scored a 37-yard return in the Big Game?']

In [23]:
p.predict(context, "Stanford")

['who did cal beat in the big game']

In [24]:
p.predict(context, "Jaydn Ott")

['Who scored the winning touchdown in the Big Game?']

In [25]:
p.predict(context, "51,892")

['How many people attended the game?']

In [14]:
p.predict(context, "125")

['How many games did Cal win in the Big Game?']

In [15]:
p.predict(context, "20-17")

['What was the score of the game that was won by the Golden Bears?']

In [16]:
p.predict(context, "20-20")

["What was the score of the game that was won by Cal's Jackson Sirmon?"]

In [17]:
p.predict(context, "three drives")

['How many drives did the Cal football team rally to win the Big Game?']

In [26]:
p.predict(context, "Jaydn Ott")   

['Who scored the winning touchdown in the Big Game?']

In [27]:
p.predict(context, "Jadyn Ott", base_model="T5", training_dataset="amalgam")

['Who scored the winning run in the 125th Big Game?']

In [28]:
p.predict(context, "Jadyn Ott", base_model="T5", training_dataset="squad")

Loading: /home/localadmin/Documents/w266_NLP_Final_Project/Models/T5_base_pt_long.squad


['Who scored the first touchdown of the game?']

In [29]:
p.predict(context, "Jadyn Ott", base_model="bart", training_dataset="squad")

Loading: /home/localadmin/Documents/w266_NLP_Final_Project/Models/bart_base_pt_long.squad


['Which Cal player scored on a 37-yard return following a wild play with two fumbles?']

In [22]:
p.predict(context, "Jadyn Ott", base_model="bart", training_dataset="triviaqa")

Loading: /home/localadmin/Documents/w266_NLP_Final_Project/Models/bart_base_pt_long.triviaqa


['Which Cal football player scored a touchdown in the fourth quarter in the Big Game?']

In [30]:
p.predict(context, "Jadyn Ott", base_model="bart", training_dataset="quac")

Loading: /home/localadmin/Documents/w266_NLP_Final_Project/Models/bart_base_pt_long.quac


['Who was the running back?']

In [24]:
p.predict(context, "Jadyn Ott", base_model="bart", training_dataset="amalgam", num_beams=10)

['Who scored the winning touchdown in the Big Game?']

In [25]:
p.predict(context, "Jadyn Ott", base_model="T5", training_dataset="amalgam", num_beams=10)

['Who scored a 1-yard run for the Golden Bears in the 125th edition of the Big Game?']

In [51]:
def model_config(model):
    print(f"""
{model.config.num_beams = } 
{model.config.no_repeat_ngram_size = } 
{model.config.max_length = } 
{model.config.min_length = }
{model.config.early_stopping = }
{model.config.model_type = }
""")
    return


In [52]:
model_config(p.retrieve_model(base_model="bart"))


model.config.num_beams = 4 
model.config.no_repeat_ngram_size = 3 
model.config.max_length = 20 
model.config.min_length = 0
model.config.early_stopping = True
model.config.model_type = 'bart'



In [60]:
model_config(p.retrieve_model(base_model="T5"))


model.config.num_beams = 1 
model.config.no_repeat_ngram_size = 0 
model.config.max_length = 20 
model.config.min_length = 0
model.config.early_stopping = False
model.config.model_type = 't5'



In [35]:
p.retrieve_model().summary

AttributeError: 'BartForConditionalGeneration' object has no attribute 'summary'